In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlalchemy
import datetime as dt
import re
import keys

In [9]:
def get_city_data(city_list):
    
    api_key = keys.open_weather_key
    cities_data = []

    for city in city_list:
        
        url = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}"
        response = requests.get(url)
        city_data = response.json()
        
        cities_dic = {
            'City': city_data['city']['name'],  
            'Country': city_data['city']['country'],  
            'Longitude': city_data['city']['coord']['lon'], 
            'Latitude': city_data['city']['coord']['lat']
        }
        cities_data.append(cities_dic)
      
    cities_df = pd.DataFrame(cities_data)

    return cities_df

In [52]:
def get_population_data(city_list=city_list):
    
  population_data = []
    
  for city in city_list:
      
    wikipedia_url = "https://www.wikipedia.org/wiki/"
    response = requests.get(wikipedia_url + city)
    city_soup = BeautifulSoup(response.content, 'html.parser')

    pop = city_soup.find(string="Population").find_next('td').get_text()
    int_pop = int(pop.replace(',',''))

    pop_data = {
        'City': city,
        'Population': int_pop,
        'Timestamp_': dt.datetime.now().strftime('%d/%m/%y')
    }
    population_data.append(pop_data)
  return pd.DataFrame(population_data)
    
get_population_data(['Berlin','Hamburg'])

,City,Population,Timestamp_
0,Berlin,3596999,28/01/25
1,Hamburg,1964021,28/01/25


In [15]:
schema = "sql_cities"
host = "127.0.0.1"
user = "root"
password = keys.connection_password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [17]:
city_list = ['Berlin', 'Hamburg', 'Munich', 'Barcelona', 'Madrid']

In [19]:
cities_df = get_city_data(city_list)

In [21]:
cities_df.to_sql('cities',
                  if_exists='append',
                  con=connection_string,
                  index=False)

5

In [23]:
cities_from_sql = pd.read_sql("cities", con=connection_string)
cities_from_sql

,city_id,City,Country,Longitude,Latitude
0,1,Berlin,DE,13.4105,52.5244
1,2,Hamburg,DE,10,53.55
2,3,Munich,DE,11.5755,48.1374
3,4,Barcelona,ES,2.159,41.3888
4,5,Madrid,ES,-3.7026,40.4165


In [27]:
def get_forecast_data(cities_from_sql=cities_from_sql):

    forecasts = []
    api_key = keys.open_weather_key

    for i, row in cities_from_sql.iterrows():
        long = row['Longitude']
        lat = row['Latitude']
    
        params = {
            'lat': lat,
            'lon': long,
            'appid': api_key
        }
        url = "https://api.openweathermap.org/data/2.5/forecast"
  
        response = requests.get(url, params=params)
        weather_data = response.json()
    
        for forecast in weather_data['list']:
            forecast_dic = {
                'city_id':row['city_id'],
                'forecast_time': forecast['dt_txt'],
                'forecast_temp': forecast['main']['temp'] - 273.15,
                'chance_of_perc': forecast.get('pop', 0),  
                'last_3hr_rain_mm': forecast.get('rain', {'3h': 0})['3h'],
                'last_3hr_snow_mm': forecast.get('snow', {'3h': 0})['3h'],
                'data_collection_time': dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            forecasts.append(forecast_dic)

    forecasts_df = pd.DataFrame(forecasts)
    forecasts_df.to_sql("forecasts", 
                        if_exists='append',
                        con=connection_string,
                        index=False)

In [30]:
forecasts_df = get_forecast_data()

In [32]:
def get_aiports_data(cities_from_sql=cities_from_sql):
    
    airports_data = [] 

    for i, row in cities_from_sql.iterrows():
    
        url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
        querystring = {
            "lat":row['Latitude'],
            "lon":row['Longitude'],
            "radiusKm":"50",
            "limit":"10",
            "withFlightInfoOnly":"true"
        }
        headers = {
            "x-rapidapi-key": keys.aero_key,
	        "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
        }
        response = requests.get(url, headers=headers, params=querystring)
        airports_json = response.json()
        
        for airport in airports_json['items']:
            airport_dic = {
                'icao': airport['icao'],
                'iata': airport['iata'],
                'airport_name': airport['name'],
            }
            airports_data.append(airport_dic)
        
    airport_df = pd.DataFrame(airports_data)
    airport_df.drop_duplicates(inplace=True)
    airport_df.to_sql("airports", 
                      if_exists='append',
                      con=connection_string,
                      index=False)

In [34]:
airport_df = get_aiports_data()

In [41]:
airports_from_sql = pd.read_sql("airports", con=connection_string)
airports_from_sql

,icao,airport_name,iata
0,EDDB,Berlin Brandenburg,BER
1,EDDH,Hamburg,HAM
2,EDDM,Munich,MUC
3,LEBL,Barcelona,BCN
4,LEMD,Madrid Adolfo Suárez –Barajas,MAD


In [43]:
def update_flights_data(cities_from_sql=cities_from_sql):
    flights_data = []
    now = dt.datetime.now()
    tomorrow = now + dt.timedelta(days=1)
    tomorrow_data = tomorrow.strftime('%Y-%m-%d')
    morning_start = f'{tomorrow_data}T00:00'
    morning_end = f'{tomorrow_data}T11:59'
    afternoon_start = f'{tomorrow_data}T12:00'
    afternoon_end = f'{tomorrow_data}T23:59'
    time_start = morning_start
    time_end = morning_end
    day_parts = [(morning_start, morning_end),(afternoon_start, afternoon_end)]
    
    for i, row in airports_from_sql.iterrows():
        
        icao = row['icao']
        
        for time_start, time_end in day_parts:
            
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{time_start}/{time_end}"

            querystring = {"withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"false"}
        
            headers = {
                "x-rapidapi-key": keys.aero_key,
	            "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
            }
            response = requests.get(url, headers=headers, params=querystring)
            flight_json = response.json()
            
            for flight in flight_json['arrivals']:
                scheduled_arrival = flight['movement']['scheduledTime']['local']
                flight_data = {
                    'flight_number': flight['number'],
                    'scheduled_time_arrival': scheduled_arrival[:-6],
                    'icao':row['icao']
                }
                flights_data.append(flight_data)

    flights_df = pd.DataFrame(flights_data)
    flights_df.to_sql("flights", 
                      if_exists='append',
                      con=connection_string,
                      index=False)

In [45]:
flights_df = update_flights_data()